In [82]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar
import time

In [83]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [147]:
#learning rate
#lr = 0.005
lr = 0.005
#number of traning steps
#num_steps =1750
#number of batch_size
#batch_size = 32


#network parameters
n_hidden_1 = 4000
#n_hidden_2 = 1000
n_hidden_2 = 4000
n_hidden_3 = 4000
#n_hidden_3 = 1200
num_input = 3072
num_classes = 10

<h1>Define Placeholder</h1>

In [148]:

tf.reset_default_graph()

#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
#drop out rate
#rate = tf.placeholder(tf.float32)
training = tf.placeholder_with_default(False, shape =(), name = 'training')
rate = 0.5

#initializer = tf.contrib.layers.variance_scaling_initializer()
initializer = tf.keras.initializers.he_normal(seed=None)
#W1 = tf.Variable(initializer([784,256]))
#Layers weight & bias
weights = {
    #'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
     'W1': tf.Variable(initializer([num_input, n_hidden_1])),
    #'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
     'W2': tf.Variable(initializer([n_hidden_1, n_hidden_2])),
    #'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
     'W3': tf.Variable(initializer([n_hidden_2, n_hidden_3])),
    'Wout': tf.Variable(tf.random_normal([n_hidden_3, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b2'),
    #'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}



<h1>Define Neural Network Architecture</h1>

In [149]:
#define a neural net model
def neural_net(x):
    
    layer_1_out = tf.add(tf.matmul(x,weights['W1']),biases['b1'])
    layer_1_out = tf.nn.relu(layer_1_out)
    drop_out = tf.layers.dropout(layer_1_out, rate, training = training)
    layer_2_out = tf.add(tf.matmul(drop_out, weights['W2']), biases['b2'])
    layer_2_out = tf.nn.relu(layer_2_out)
    drop_out = tf.layers.dropout(layer_2_out, rate, training = training)
    layer_3_out = tf.add(tf.matmul(drop_out, weights['W3']), biases['b3'])
    layer_3_out = tf.nn.relu(layer_3_out)
    drop_out = tf.layers.dropout(layer_3_out, rate, training = training)
    out = tf.add(tf.matmul(drop_out, weights['Wout']),biases['bout'])
    return out

<h1>Define cost andoptimization</h1>

In [150]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [151]:
valid_dict = pickle.load(open('processed_validate','rb'))
train_dict1 = pickle.load(open('processed_train1','rb'))
train_dict2 = pickle.load(open('processed_train2','rb'))
train_dict3 = pickle.load(open('processed_train3','rb'))
train_dict4 = pickle.load(open('processed_train4','rb'))
train_dict5 = pickle.load(open('processed_train5','rb'))
test_dict  = pickle.load(open('processed_test','rb'))

valid_features = valid_dict['normal feature']
valid_labels   = valid_dict['one hot labels']

train_features1 = train_dict1['normal feature']
train_labels1   = train_dict1['one hot labels']

train_features2 = train_dict2['normal feature']
train_labels2   = train_dict2['one hot labels']

train_features3 = train_dict3['normal feature']
train_labels3   = train_dict3['one hot labels']

train_features4 = train_dict4['normal feature']
train_labels4   = train_dict4['one hot labels']

train_features5 = train_dict5['normal feature']
train_labels5   = train_dict5['one hot labels']

test_features = test_dict['normal feature']
test_labels   = test_dict['one hot labels']

#Put all the training data together:
totalX = np.concatenate((train_features1,train_features2,train_features3,train_features4,train_features5))
totalY = np.concatenate((train_labels1,train_labels2,train_labels3,train_labels4,train_labels5))


In [152]:
with tf.Session() as sess:
    sess.run(init)
    
    num_steps = 390
    batch_size = 128
    #train with the first training batch
    #for epoch in range(10)
    for epoch in range(1000):
        start_time = time.time()
        for i in range(num_steps):
            #fetch batch
            #The input values have already been normalized
            batch_x_train = totalX[i*batch_size:(i+1)*batch_size,:]
            batch_y_train = totalY[i*batch_size:(i+1)*batch_size,:]

            #batch_x = train_features1[0:1028,:]
            #print(batch_size)
            #print(np.shape(batch_x_train))
            #run optimization
            sess.run(train_op, feed_dict={X:batch_x_train, Y:batch_y_train, training:True})
            #if i % 100 ==0:
            #acc = sess.run(accuracy,feed_dict={X:batch_x_train, Y:batch_y_train, rate:0.5})
            #print("In training batch: step "+str(i)+", Accuracy= {:.3f}".format(acc))
                #print(mnist.train.shape())
            if i % 100 ==0:
                #no drop out in validation
                print("Validating Accuracy:", sess.run(accuracy, feed_dict={X:valid_features, Y:valid_labels}))
                #print(sess.run(weights['W1'], feed_dict={X:valid_features, Y:valid_labels}))
        
        print("epoch {} is finished".format(epoch))
        elapsed_time = time.time() - start_time
        print("time used in the epoch is {}".format(elapsed_time))
        if epoch % 10 == 0:
            #no drop out in testing
            print("Testing Accuracy is :", sess.run(accuracy, feed_dict={X:test_features, Y:test_labels} ))
    print("Final testing accuracy is :", sess.run(accuracy, feed_dict={X:test_features, Y:test_labels} ))

Validating Accuracy: 0.1038
Validating Accuracy: 0.1246
Validating Accuracy: 0.119
Validating Accuracy: 0.1196
epoch 0 is finished
time used in the epoch is 50.15264081954956
Testing Accuracy is : 0.1315
Validating Accuracy: 0.137
Validating Accuracy: 0.1286
Validating Accuracy: 0.1244
Validating Accuracy: 0.1196
epoch 1 is finished
time used in the epoch is 49.29786205291748
Validating Accuracy: 0.127
Validating Accuracy: 0.1204
Validating Accuracy: 0.1296
Validating Accuracy: 0.1388
epoch 2 is finished
time used in the epoch is 49.3638231754303
Validating Accuracy: 0.1376
Validating Accuracy: 0.1372
Validating Accuracy: 0.1218
Validating Accuracy: 0.1618
epoch 3 is finished
time used in the epoch is 49.63666844367981
Validating Accuracy: 0.1596
Validating Accuracy: 0.1608
Validating Accuracy: 0.1512
Validating Accuracy: 0.1296
epoch 4 is finished
time used in the epoch is 49.506742000579834
Validating Accuracy: 0.1446
Validating Accuracy: 0.173
Validating Accuracy: 0.1696
Validating 

Validating Accuracy: 0.368
Validating Accuracy: 0.3764
Validating Accuracy: 0.3778
Validating Accuracy: 0.3764
epoch 46 is finished
time used in the epoch is 49.49874711036682
Validating Accuracy: 0.3746
Validating Accuracy: 0.367
Validating Accuracy: 0.3684
Validating Accuracy: 0.3848
epoch 47 is finished
time used in the epoch is 49.370819330215454
Validating Accuracy: 0.3684
Validating Accuracy: 0.3812
Validating Accuracy: 0.3808
Validating Accuracy: 0.3866
epoch 48 is finished
time used in the epoch is 49.68464112281799
Validating Accuracy: 0.38
Validating Accuracy: 0.3758
Validating Accuracy: 0.3972
Validating Accuracy: 0.385
epoch 49 is finished
time used in the epoch is 49.339837312698364
Validating Accuracy: 0.3882
Validating Accuracy: 0.3592
Validating Accuracy: 0.3894
Validating Accuracy: 0.3762
epoch 50 is finished
time used in the epoch is 49.509740591049194
Testing Accuracy is : 0.3656
Validating Accuracy: 0.39
Validating Accuracy: 0.382
Validating Accuracy: 0.37
Validatin

epoch 91 is finished
time used in the epoch is 49.42878484725952
Validating Accuracy: 0.4602
Validating Accuracy: 0.456
Validating Accuracy: 0.4516
Validating Accuracy: 0.4648
epoch 92 is finished
time used in the epoch is 49.22390389442444
Validating Accuracy: 0.4434
Validating Accuracy: 0.469
Validating Accuracy: 0.4538
Validating Accuracy: 0.4716
epoch 93 is finished
time used in the epoch is 49.6256742477417
Validating Accuracy: 0.4566
Validating Accuracy: 0.4688
Validating Accuracy: 0.454
Validating Accuracy: 0.4618
epoch 94 is finished
time used in the epoch is 49.36682200431824
Validating Accuracy: 0.4514
Validating Accuracy: 0.4666
Validating Accuracy: 0.4566
Validating Accuracy: 0.4514
epoch 95 is finished
time used in the epoch is 49.53472638130188
Validating Accuracy: 0.4578
Validating Accuracy: 0.4682
Validating Accuracy: 0.4378
Validating Accuracy: 0.4808
epoch 96 is finished
time used in the epoch is 49.321852684020996
Validating Accuracy: 0.4608
Validating Accuracy: 0.46

Validating Accuracy: 0.5114
Validating Accuracy: 0.5068
epoch 137 is finished
time used in the epoch is 49.48675298690796
Validating Accuracy: 0.5218
Validating Accuracy: 0.5154
Validating Accuracy: 0.5108
Validating Accuracy: 0.5176
epoch 138 is finished
time used in the epoch is 49.59469151496887
Validating Accuracy: 0.507
Validating Accuracy: 0.524
Validating Accuracy: 0.4872
Validating Accuracy: 0.512
epoch 139 is finished
time used in the epoch is 49.35782766342163
Validating Accuracy: 0.5176
Validating Accuracy: 0.5148
Validating Accuracy: 0.499
Validating Accuracy: 0.5186
epoch 140 is finished
time used in the epoch is 49.37481760978699
Testing Accuracy is : 0.449
Validating Accuracy: 0.5
Validating Accuracy: 0.5118
Validating Accuracy: 0.4996
Validating Accuracy: 0.5236
epoch 141 is finished
time used in the epoch is 49.38680934906006
Validating Accuracy: 0.5124
Validating Accuracy: 0.5144
Validating Accuracy: 0.5158
Validating Accuracy: 0.5172
epoch 142 is finished
time used i

Validating Accuracy: 0.5578
Validating Accuracy: 0.5504
Validating Accuracy: 0.5384
Validating Accuracy: 0.561
epoch 183 is finished
time used in the epoch is 49.40380096435547
Validating Accuracy: 0.537
Validating Accuracy: 0.553
Validating Accuracy: 0.5372
Validating Accuracy: 0.5626
epoch 184 is finished
time used in the epoch is 49.51673722267151
Validating Accuracy: 0.5618
Validating Accuracy: 0.5494
Validating Accuracy: 0.5414
Validating Accuracy: 0.5586
epoch 185 is finished
time used in the epoch is 49.352829456329346
Validating Accuracy: 0.5452
Validating Accuracy: 0.5492
Validating Accuracy: 0.5284
Validating Accuracy: 0.5622
epoch 186 is finished
time used in the epoch is 49.40380072593689
Validating Accuracy: 0.5474
Validating Accuracy: 0.5618
Validating Accuracy: 0.5494
Validating Accuracy: 0.5642
epoch 187 is finished
time used in the epoch is 49.34383487701416
Validating Accuracy: 0.5584
Validating Accuracy: 0.5542
Validating Accuracy: 0.5526
Validating Accuracy: 0.5608


...

Validating Accuracy: 0.8564
epoch 775 is finished
time used in the epoch is 49.55371594429016
Validating Accuracy: 0.8508
Validating Accuracy: 0.8496
Validating Accuracy: 0.8434
Validating Accuracy: 0.8576
epoch 776 is finished
time used in the epoch is 49.4337842464447
Validating Accuracy: 0.8534
Validating Accuracy: 0.8444
Validating Accuracy: 0.8456
Validating Accuracy: 0.8594
epoch 777 is finished
time used in the epoch is 49.4507737159729
Validating Accuracy: 0.8538
Validating Accuracy: 0.8392
Validating Accuracy: 0.8512
Validating Accuracy: 0.841
epoch 778 is finished
time used in the epoch is 49.37981414794922
Validating Accuracy: 0.8386
Validating Accuracy: 0.8572
Validating Accuracy: 0.8486
Validating Accuracy: 0.8382
epoch 779 is finished
time used in the epoch is 49.44577717781067
Validating Accuracy: 0.853
Validating Accuracy: 0.8608
Validating Accuracy: 0.8466
Validating Accuracy: 0.8446
epoch 780 is finished
time used in the epoch is 49.38481068611145
Testing Accuracy is 

Validating Accuracy: 0.8598
Validating Accuracy: 0.8302
Validating Accuracy: 0.8512
Validating Accuracy: 0.8626
epoch 821 is finished
time used in the epoch is 49.39480638504028
Validating Accuracy: 0.8686
Validating Accuracy: 0.858
Validating Accuracy: 0.8544
Validating Accuracy: 0.8574
epoch 822 is finished
time used in the epoch is 49.197918176651
Validating Accuracy: 0.8624
Validating Accuracy: 0.864
Validating Accuracy: 0.857
Validating Accuracy: 0.8634
epoch 823 is finished
time used in the epoch is 49.56670832633972
Validating Accuracy: 0.85
Validating Accuracy: 0.861
Validating Accuracy: 0.861
Validating Accuracy: 0.856
epoch 824 is finished
time used in the epoch is 49.45377230644226
Validating Accuracy: 0.8702
Validating Accuracy: 0.8776
Validating Accuracy: 0.8746
Validating Accuracy: 0.8572
epoch 825 is finished
time used in the epoch is 49.42978644371033
Validating Accuracy: 0.8548
Validating Accuracy: 0.8682
Validating Accuracy: 0.871
Validating Accuracy: 0.8604
epoch 826

epoch 866 is finished
time used in the epoch is 49.770592212677
Validating Accuracy: 0.8748
Validating Accuracy: 0.878
Validating Accuracy: 0.8586
Validating Accuracy: 0.8754
epoch 867 is finished
time used in the epoch is 49.8305561542511
Validating Accuracy: 0.875
Validating Accuracy: 0.8802
Validating Accuracy: 0.8558
Validating Accuracy: 0.865
epoch 868 is finished
time used in the epoch is 49.452773332595825
Validating Accuracy: 0.8672
Validating Accuracy: 0.8786
Validating Accuracy: 0.8816
Validating Accuracy: 0.871
epoch 869 is finished
time used in the epoch is 49.32684874534607
Validating Accuracy: 0.8854
Validating Accuracy: 0.8832
Validating Accuracy: 0.8774
Validating Accuracy: 0.865
epoch 870 is finished
time used in the epoch is 49.23889517784119
Testing Accuracy is : 0.5228
Validating Accuracy: 0.8752
Validating Accuracy: 0.8662
Validating Accuracy: 0.8722
Validating Accuracy: 0.8786
epoch 871 is finished
time used in the epoch is 49.280871868133545
Validating Accuracy: 

Validating Accuracy: 0.8934
Validating Accuracy: 0.8808
Validating Accuracy: 0.8732
Validating Accuracy: 0.8926
epoch 912 is finished
time used in the epoch is 49.326844215393066
Validating Accuracy: 0.9012
Validating Accuracy: 0.8808
Validating Accuracy: 0.8804
Validating Accuracy: 0.8868
epoch 913 is finished
time used in the epoch is 49.355828285217285
Validating Accuracy: 0.883
Validating Accuracy: 0.8846
Validating Accuracy: 0.8894
Validating Accuracy: 0.8742
epoch 914 is finished
time used in the epoch is 49.35482907295227
Validating Accuracy: 0.8866
Validating Accuracy: 0.8848
Validating Accuracy: 0.8718
Validating Accuracy: 0.879
epoch 915 is finished
time used in the epoch is 49.269877195358276
Validating Accuracy: 0.8988
Validating Accuracy: 0.8838
Validating Accuracy: 0.8828
Validating Accuracy: 0.8822
epoch 916 is finished
time used in the epoch is 49.091979026794434
Validating Accuracy: 0.8788
Validating Accuracy: 0.8658
Validating Accuracy: 0.8908
Validating Accuracy: 0.8

epoch 957 is finished
time used in the epoch is 49.19192171096802
Validating Accuracy: 0.8958
Validating Accuracy: 0.8782
Validating Accuracy: 0.8848
Validating Accuracy: 0.8952
epoch 958 is finished
time used in the epoch is 49.33284139633179
Validating Accuracy: 0.9006
Validating Accuracy: 0.8926
Validating Accuracy: 0.8886
Validating Accuracy: 0.8886
epoch 959 is finished
time used in the epoch is 48.9810426235199
Validating Accuracy: 0.91
Validating Accuracy: 0.9042
Validating Accuracy: 0.8802
Validating Accuracy: 0.8874
epoch 960 is finished
time used in the epoch is 49.45077395439148
Testing Accuracy is : 0.5186
Validating Accuracy: 0.8924
Validating Accuracy: 0.8908
Validating Accuracy: 0.8874
Validating Accuracy: 0.879
epoch 961 is finished
time used in the epoch is 49.281869888305664
Validating Accuracy: 0.886
Validating Accuracy: 0.884
Validating Accuracy: 0.8948
Validating Accuracy: 0.897
epoch 962 is finished
time used in the epoch is 49.280871868133545
Validating Accuracy: